In [1]:
import streamlit as st
import pandas as pd


In [64]:
nested_egg = pd.read_json('https://api.flipsidecrypto.com/api/v2/queries/290a91fb-8186-486d-992e-985839352809/data/latest')
loot = pd.read_json('https://api.flipsidecrypto.com/api/v2/queries/2208cc91-e624-4b40-b666-f65c8b6405f8/data/latest')
eggs = pd.read_json('https://api.flipsidecrypto.com/api/v2/queries/ee319c5f-4f03-41b5-b607-99f306fe1e2f/data/latest')
meteor_dust = pd.read_json('https://api.flipsidecrypto.com/api/v2/queries/8eebec3c-db0e-4051-a5c2-50a5deb1070b/data/latest')
meteors = pd.read_json('https://api.flipsidecrypto.com/api/v2/queries/b9778a1d-9b63-4ba5-a48f-7b28569b5603/data/latest')
#IT IS ABSOLUTELY CRUCIAL THAT YOU DO NOT CHANGE THE ORIGINAL NAMES OF THE DATAFRAME COLUMNS ON VELOCITY

#MUST STORE IN A DICTIONARY 
current_nft_dict = {'nested_egg':nested_egg, 'loot':loot, 'eggs':eggs, 'meteor_dust':meteor_dust, 'meteors':meteors}



In [ ]:
nft_luna_price_df = []
for nft in current_nft_dict:
    #renames columns from Flipside dataframes according to NFT
    nft_luna_price_df.append(current_nft_dict[nft][['BLOCK_TIMESTAMP', 'NFT_LUNA_PRICE']].rename({'NFT_LUNA_PRICE':f'{nft}'}, axis = 1))
nft_luna_price_df = pd.concat(nft_luna_price_df)
nft_luna_price_df['BLOCK_TIMESTAMP'] = pd.to_datetime(nft_luna_price_df['BLOCK_TIMESTAMP'])
nft_luna_price_df.set_index('BLOCK_TIMESTAMP', inplace = True)
nft_luna_price_df.index = nft_luna_price_df.index.round('D')

# START HERE

In [99]:
nft_luna_price_df = []
for nft in current_nft_dict:
    #renames columns from Flipside dataframes according to NFT
    nft_luna_price_df.append(current_nft_dict[nft][['BLOCK_TIMESTAMP', 'NFT_TYPE', 'NFT_LUNA_PRICE', 'NFT_UST_PRICE_AT_PURCHASE']])

nft_luna_price_df = pd.concat(nft_luna_price_df)
nft_luna_price_df['BLOCK_TIMESTAMP'] = pd.to_datetime(nft_luna_price_df['BLOCK_TIMESTAMP'])
nft_luna_price_df.set_index('BLOCK_TIMESTAMP', inplace = True)
nft_luna_price_df.index = nft_luna_price_df.index.round('D')
nft_luna_price_df


,NFT_TYPE,NFT_LUNA_PRICE,NFT_UST_PRICE_AT_PURCHASE
BLOCK_TIMESTAMP,,,
2022-01-30,Levana Nested Egg,5.75,291.799528
2022-01-30,Levana Nested Egg,5.75,291.799528
2022-01-30,Levana Nested Egg,5.75,298.984103
2022-01-30,Levana Nested Egg,12.50,642.477332
2022-01-30,Levana Nested Egg,6.50,335.149540
...,...,...,...
2021-11-11,Levana Meteors,0.99,51.273975
2021-11-11,Levana Meteors,10.00,517.918941
2021-11-11,Levana Meteors,0.80,41.433515


In [122]:
# master_df = nft_luna_price_df.groupby(['BLOCK_TIMESTAMP', 'NFT_TYPE']).sum().rename(columns={'NFT_LUNA_PRICE':'TOTAL_LUNA', 'NFT_UST_PRICE_AT_PURCHASE':'TOTAL_UST'})
# master_df

# master_df = nft_luna_price_df.groupby(['BLOCK_TIMESTAMP', 'NFT_TYPE']).mean().rename(columns={'NFT_LUNA_PRICE':'AVERAGE_LUNA', 'NFT_UST_PRICE_AT_PURCHASE':'AVERAGE_UST'})
# master_df

# master_df = nft_luna_price_df.groupby(['BLOCK_TIMESTAMP', 'NFT_TYPE']).count().rename(columns={'NFT_LUNA_PRICE':'TRANSACTION_COUNT'})
# master_df

#master_df = nft_luna_price_df.groupby(['BLOCK_TIMESTAMP', 'NFT_TYPE']).count().rename(columns={'NFT_LUNA_PRICE':'TRANSACTION_COUNT'})
#master_df

# master_df = nft_luna_price_df.groupby(['BLOCK_TIMESTAMP', 'NFT_TYPE']).min().rename(columns={'NFT_LUNA_PRICE':'MIN_LUNA', 'NFT_UST_PRICE_AT_PURCHASE':'MIN_UST'})
# master_df

# master_df = nft_luna_price_df.groupby(['BLOCK_TIMESTAMP', 'NFT_TYPE']).max().rename(columns={'NFT_LUNA_PRICE':'MAX_LUNA', 'NFT_UST_PRICE_AT_PURCHASE':'MAX_UST'})
# master_df

#master_df = nft_luna_price_df.groupby(['BLOCK_TIMESTAMP', 'NFT_TYPE']).cumsum()

ValueError: Grouper for 'BLOCK_TIMESTAMP' not 1-dimensional

In [116]:
nft_luna_price_df = []
for nft in current_nft_dict:
    #renames columns from Flipside dataframes according to NFT
    nft_luna_price_df.append(current_nft_dict[nft][['BLOCK_TIMESTAMP', 'NFT_TYPE', 'NFT_LUNA_PRICE']].rename({'NFT_LUNA_PRICE':f'{nft}_LUNA_PRICE'}, axis = 1))
nft_luna_price_df = pd.concat(nft_luna_price_df, axis= 1)
nft_luna_price_df['BLOCK_TIMESTAMP'] = pd.to_datetime(nft_luna_price_df['BLOCK_TIMESTAMP'])
nft_luna_price_df.set_index('BLOCK_TIMESTAMP', inplace = True)
nft_luna_price_df.index = nft_luna_price_df.index.round('D')
nft_luna_price_df.groupby('BLOCK_TIMESTAMP').sum()

ValueError: cannot assemble with duplicate keys

In [63]:
nft_ust_price_df = []
for nft in current_nft_dict:
    #renames columns from Flipside dataframes according to NFT
    nft_ust_price_df.append(current_nft_dict[nft][['BLOCK_TIMESTAMP', 'NFT_UST_PRICE_AT_PURCHASE']].rename({'NFT_UST_PRICE_AT_PURCHASE':f'{nft}_UST_PRICE'}, axis = 1))
nft_ust_price_df = pd.concat(nft_ust_price_df)
nft_ust_price_df['BLOCK_TIMESTAMP'] = pd.to_datetime(nft_ust_price_df['BLOCK_TIMESTAMP'])
nft_ust_price_df.set_index('BLOCK_TIMESTAMP', inplace = True)
nft_ust_price_df.index = nft_ust_price_df.index.round('D')
nft_ust_price_df.groupby('BLOCK_TIMESTAMP').sum()

,nested_egg_UST_PRICE,loot_UST_PRICE,eggs_UST_PRICE,meteor_dust_UST_PRICE,meteors_UST_PRICE
BLOCK_TIMESTAMP,,,,,
2021-11-11,0.000000,0.000000,0.000000,0.000000,36529.469834
2021-11-12,0.000000,0.000000,0.000000,0.000000,97246.288728
2021-11-13,0.000000,0.000000,0.000000,0.000000,7886.429952
2021-11-14,0.000000,0.000000,0.000000,0.000000,19818.606276
2021-11-15,0.000000,0.000000,0.000000,0.000000,28926.395622
...,...,...,...,...,...
2022-01-26,26932.261334,2424.284101,7626.571364,15962.438021,4499.899561
2022-01-27,9236.058513,1935.495190,10161.408364,10305.208151,13978.989361
2022-01-28,0.000000,3466.615553,4442.340503,6296.279094,4790.275722


In [51]:
#pd.DataFrame(nft_ust_price_df.sum()).reset_index().rename(columns={0:"UST Exchanged Sum"})
nft_ust_price_df.sum()

nested_egg_NFT_UST_PRICE_AT_PURCHASE     3.160115e+05
loot_NFT_UST_PRICE_AT_PURCHASE           1.401086e+05
eggs_NFT_UST_PRICE_AT_PURCHASE           1.170355e+06
meteor_dust_NFT_UST_PRICE_AT_PURCHASE    7.265926e+05
meteors_NFT_UST_PRICE_AT_PURCHASE        1.181229e+06
dtype: float64

In [60]:
tx_count = pd.DataFrame(nft_ust_price_df.apply(lambda x: (x > 0).sum(), axis =  1).groupby('BLOCK_TIMESTAMP').sum()).reset_index()
pd.DataFrame(tx_count.rename(columns={0:"Transaction Count"})['Transaction Count'])

,Transaction Count
0,267
1,247
2,99
3,78
4,195
...,...
76,210
77,176
78,216
79,85


In [52]:
nft_luna_price_df.index = nft_luna_price_df.index.round('H')
nft_luna_price_df.groupby('BLOCK_TIMESTAMP').sum()

NameError: name 'nft_luna_price_df' is not defined